## Workshop Week 6

- Student ID **46199551**
- Student Name **Aditya Sharma**

## Logistic Regression
Breast Cancer data from [the UCI repository](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains records corresponding to 
cases of observed tumors.   There are a number of observations for each and a categorisation in the `class` column: 2 for benign (good), 4 for malignant (bad).  Your task is to build a logistic regression model to classify these cases. 

The data is provided as a CSV file.  There are a small number of cases where no value is available, these are indicated in the data with `?`. I have used the `na_values` keyword for `read_csv` to have these interpreted as `NaN` (Not a Number).  Your first task is to decide what to do with these rows. You could just drop these rows or you could [impute them from the other data](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).

You then need to follow the procedure outlined in the lecture for generating a train/test set, building and evaluating a model. Your goal is to build the best model possible over this data.   Your first step should be to build a logistic regression model using all of the features that are available.
  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

In [2]:
bcancer = pd.read_csv("files/breast-cancer-wisconsin.csv", na_values="?")
bcancer.head()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [3]:
bcancer.shape



(699, 11)

In [4]:
bcancer.describe()   # Look at the statistical summary of the dataframe

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,683.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.544656,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,3.643857,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [5]:
set(bcancer['class'])
# Check how many classes we do have from the "class" column , 2 means no cancer and 4 means cancer

{2, 4}

In [6]:
print('No of samples in class 2', bcancer[bcancer['class']==4].shape[0])
print('No of samples in class 2', bcancer[bcancer['class']==2].shape[0])# Check number of samples for each class and comment whether dataset is balanced?

No of samples in class 2 241
No of samples in class 2 458


In [7]:
bcancer.isna().sum()
# Deal with the NaN values in the data

sample_code_number              0
clump_thickness                 0
uniformity_cell_size            0
uniformity_cell_shape           0
marginal_adhesion               0
single_epithelial_cell_size     0
bare_nuclei                    16
bland_chromatin                 0
normal_nucleoli                 0
mitoses                         0
class                           0
dtype: int64

In [8]:
bcancer=bcancer.dropna() 
print(bcancer.shape)
print(bcancer.isna().sum())# To drop NAn values in the dataset by dropping rows

(683, 11)
sample_code_number             0
clump_thickness                0
uniformity_cell_size           0
uniformity_cell_shape          0
marginal_adhesion              0
single_epithelial_cell_size    0
bare_nuclei                    0
bland_chromatin                0
normal_nucleoli                0
mitoses                        0
class                          0
dtype: int64


In [9]:
train, test = train_test_split(bcancer, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)
# Split your data into training(80%) and testing data (20%) and use random_state=142

(546, 11)
(137, 11)


In [10]:
#sns.pairplot(bcancer, hue='class')

In [19]:
X_train = train.drop(['sample_code_number' , 'class'],axis=1)  #Using all colums for features except these 2
y_train = train['class']
X_test = test.drop(['sample_code_number', 'class'],axis=1)
y_test = test['class']
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Build your Logistic Regression model

(546, 9)
(546,)
(137, 9)
(137,)


In [20]:
model=LogisticRegression()
model.fit(X_train,y_train)

    # Do predictions on test set

LogisticRegression()

In [21]:
y_test_prediction=model.predict(X_test)
y_test_prediction

array([2, 4, 2, 2, 4, 4, 2, 4, 2, 4, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 4, 4, 4, 2, 2, 4, 2, 2, 2, 4, 2, 4,
       2, 4, 2, 4, 2, 2, 2, 4, 2, 2, 4, 2, 4, 4, 4, 2, 2, 4, 2, 2, 4, 2,
       4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 4, 2, 2, 4,
       2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 4, 4, 2, 2, 2, 2, 2, 4, 2,
       2, 4, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 4, 4, 2, 2, 4, 2, 4, 2, 2, 2,
       2, 4, 2, 4, 4], dtype=int64)

In [22]:
y_test

280    2
232    2
369    2
563    2
491    4
      ..
645    2
353    4
307    2
126    4
67     4
Name: class, Length: 137, dtype: int64

### Evaluation

To evaluate a classification model we want to look at how many cases were correctly classified and how many
were in error.  In this case we have two outcomes - benign and malignant.   SKlearn has some useful tools, the 
[accuracy_score]() function gives a score from 0-1 for the proportion correct.  The 
[confusion_matrix](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function 
shows how many were classified correctly and what errors were made.  Use these to summarise the performance of 
your model (these functions have already been imported above).

In [23]:
print('accuracy score for the sample is ',accuracy_score(y_test,y_test_prediction))# Evaluate the performance of your trained model


accuracy score for the sample is  0.9635036496350365


In [24]:
print(confusion_matrix(y_test,y_test_prediction) )

[[83  2]
 [ 3 49]]


**This is the checkpoint mark for this week's workshop. You need to report `Accuracy Score` on test set and also show `confusion matrix`. You also need to provide analysis based on the results you got.**

### Feature Selection

Since you have many features available, one part of building the best model will be to select which features to use as input to the classifier. Your initial model used all of the features but it is possible that a better model can 
be built by leaving some of them out.   Test this by building a few models with subsets of the features - how do your models perform? 

This process can be automated.  The [sklearn RFE function](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination) implements __Recursive Feature Estimation__ which removes 
features one by one, evaluating the model each time and selecting the best model for a target number of features.  Use RFE to select features for a model with 3, 4 and 5 features - can you build a model that is as good or better than your initial model?

## Conclusion

Write a brief conclusion to your experiment.  You might comment on the proportion of __false positive__ and __false negative__ classifications your model makes.  How useful would this model be in a clinical diagnostic setting? 

## Commit your finished work on Github
Here are the list of steps you need to follow to commit your work on Github to get checkpoint mark for this week.

Once you finished all the above questions, save the notebook by clicking 'save' button in the toolbar.

You need to follow the same instructions to commit your work on your Github repository.

Step 1. Change your current directory to `practical-workshops-yourName` by doing `cd` command. You can type:
                    `cd practical-workshops-yourName`
                    
Step 2: Add your Workshop Week 6.ipynb using:
                 `git add "Workshop Week 6.ipynb`
                 
Step 3: Commit your work:
                `git commit -m "Finished Workshop 6`
                
Step 4: Push your changes:
                `git push origin main`
                
Step 5: Confirm whether your finished work is now on Github repository by signing into your Github account and clicking on your repository. You can see your added `Workshop Week 6.ipynb` file as well as your `commit message` and `time` of your commit.

Step 6: Well done!. You have now finished your Practical Workshop Week 6. Appreciate yourself.